In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path= '/content/drive/MyDrive/VOCtrainval_14-Jul-2008.tar'




import tarfile
with tarfile.open(path, 'r') as tar:
    tar.extractall()


# Seprating the images into their respective class folders

In [ ]:
import shutil
import os


folder_path = "/content/VOCdevkit/VOC2008/JPEGImages"

def extract_positive_images(directory, file_name):
    positive_images = []
    variations = [
        file_name,
        file_name.replace("_train", "_trainval"),
        file_name.replace("_train", "_val")
    ]

    for variation in variations:
        file_path = os.path.join(directory, variation)
        with open(file_path, "r") as file:
            lines = file.readlines()

        for line in lines:
            if line.strip().endswith(" 1"):
                image_name = line.split()[0] + ".jpg"
                if image_name not in positive_images:
                    positive_images.append(image_name)

    return positive_images


data_directory = "/content/VOCdevkit/VOC2008/ImageSets/Main/"


aeroplane_images = extract_positive_images(data_directory, "aeroplane_train.txt")
bicycle_images = extract_positive_images(data_directory, "bicycle_train.txt")
bird_images = extract_positive_images(data_directory, "bird_train.txt")
boat_images = extract_positive_images(data_directory, "boat_train.txt")
bottle_images = extract_positive_images(data_directory, "bottle_train.txt")
bus_images = extract_positive_images(data_directory, "bus_train.txt")
car_images = extract_positive_images(data_directory, "car_train.txt")
cat_images = extract_positive_images(data_directory, "cat_train.txt")
chair_images = extract_positive_images(data_directory, "chair_train.txt")
cow_images = extract_positive_images(data_directory, "cow_train.txt")
diningtable_images = extract_positive_images(data_directory, "diningtable_train.txt")
dog_images = extract_positive_images(data_directory, "dog_train.txt")
horse_images = extract_positive_images(data_directory, "horse_train.txt")
motorbike_images = extract_positive_images(data_directory, "motorbike_train.txt")
person_images = extract_positive_images(data_directory, "person_train.txt")
pottedplant_images = extract_positive_images(data_directory, "pottedplant_train.txt")
sheep_images = extract_positive_images(data_directory, "sheep_train.txt")
sofa_images = extract_positive_images(data_directory, "sofa_train.txt")
train_images = extract_positive_images(data_directory, "train_train.txt")
tvmonitor_images = extract_positive_images(data_directory, "tvmonitor_train.txt")

jpg_files = []

for file in os.listdir(folder_path):
    if file.endswith(".jpg"):
        jpg_files.append(file)


In [ ]:
len(aeroplane_images)

236

In [ ]:
import shutil
import os


class_data_lists = {
    "Aeroplane": aeroplane_images,
    "Bicycle": bicycle_images,
    "Bird": bird_images,
    "Boat": boat_images,
    "Bottle": bottle_images,
    "Bus": bus_images,
    "Car": car_images,
    "Cat": cat_images,
    "Chair": chair_images,
    "Cow": cow_images,
    "Diningtable": diningtable_images,
    "Dog": dog_images,
    "Horse": horse_images,
    "Motorbike": motorbike_images,
    "Person": person_images,
    "Pottedplant": pottedplant_images,
    "Sheep": sheep_images,
    "Sofa": sofa_images,
    "Train": train_images,
    "Tvmonitor": tvmonitor_images
}


folder_path = "/content/VOCdevkit/VOC2008/JPEGImages"





#Splitting the images into train and test split

In [ ]:
from sklearn.model_selection import train_test_split
train_data_lists = {class_name: [] for class_name in class_data_lists}
test_data_lists = {class_name: [] for class_name in class_data_lists}


for class_name, data_list in class_data_lists.items():
    train_images, test_images = train_test_split(data_list, test_size=0.2, random_state=42)
    train_data_lists[class_name] = train_images
    test_data_lists[class_name] = test_images


for class_name, train_images in train_data_lists.items():
    for file in train_images:
        try:
            src_path = os.path.join(folder_path, file)
            dest_dir = os.path.join("train", class_name)
            dest_path = os.path.join(dest_dir, file)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copyfile(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {file} to {dest_dir}: {e}")


for class_name, test_images in test_data_lists.items():
    for file in test_images:
        try:
            src_path = os.path.join(folder_path, file)
            dest_dir = os.path.join("test", class_name)
            dest_path = os.path.join(dest_dir, file)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copyfile(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {file} to {dest_dir}: {e}")

print("Images divided into Train and Test data Successfully")

Images divided into Train and Test data Successfully


In [ ]:
import os



train_folder_path = "/content/train/Aeroplane"
test_folder_path = "/content/test/Aeroplane"


aeroplane_train_images = os.listdir(train_folder_path)
aeroplane_test_images = os.listdir(test_folder_path)


num_aeroplane_train_images = len(aeroplane_train_images)

num_aeroplane_test_images = len(aeroplane_test_images)

print(f"Number of Aeroplane images in the train dataset: {num_aeroplane_train_images}")
print(f"Number of Aeroplane images in the test dataset: {num_aeroplane_test_images}")
print(f"Total number of images : {num_aeroplane_train_images + num_aeroplane_test_images}")


Number of Aeroplane images in the train dataset: 188
Number of Aeroplane images in the test dataset: 48
Total number of images : 236


# Preparing the data for training

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import os

train_path = "/content/train"
test_path = "/content/test"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
print("Class Indices:", train_generator.class_indices)

Found 4907 images belonging to 20 classes.
Found 1236 images belonging to 20 classes.
Class Indices: {'Aeroplane': 0, 'Bicycle': 1, 'Bird': 2, 'Boat': 3, 'Bottle': 4, 'Bus': 5, 'Car': 6, 'Cat': 7, 'Chair': 8, 'Cow': 9, 'Diningtable': 10, 'Dog': 11, 'Horse': 12, 'Motorbike': 13, 'Person': 14, 'Pottedplant': 15, 'Sheep': 16, 'Sofa': 17, 'Train': 18, 'Tvmonitor': 19}


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in base_model.layers:  # Freeze the convolutional layers
    layer.trainable = False


x = Flatten()(base_model.output)S
x = Dense(256, activation='relu')(x)
x = Dense(len(class_data_lists), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

58889256/58889256 [==============================] - 0s 0us/step


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size)


model.save('vgg_model.h5')


<ipython-input-10-ce70b97cb3c7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
153/153 [==============================] - 94s 549ms/step - loss: 2.4066 - accuracy: 0.3719 - val_loss: 1.9766 - val_accuracy: 0.4137
Epoch 2/20
153/153 [==============================] - 80s 521ms/step - loss: 1.7351 - accuracy: 0.4699 - val_loss: 1.9081 - val_accuracy: 0.4276
Epoch 3/20
153/153 [==============================] - 79s 515ms/step - loss: 1.5448 - accuracy: 0.5102 - val_loss: 1.9406 - val_accuracy: 0.4178
Epoch 4/20
153/153 [==============================] - 79s 514ms/step - loss: 1.4016 - accuracy: 0.5417 - val_loss: 1.9248 - val_accuracy: 0.4194
Epoch 5/20
153/153 [==============================] - 82s 539ms/step - loss: 1.3276 - accuracy: 0.5499 - val_loss: 1.9484 - val_accuracy: 0.3980
Epoch 6/20
153/153 [==============================] - 78s 510ms/step - loss: 1.2552 - accuracy: 0.5643 - val_loss: 2.0448 - val_accuracy: 0.3890
Epoch 7/20
153/153 [==============================] - 79s 516ms/step - loss: 1.1857 - accuracy: 0.5869 - val_loss: 2.0491 - val_ac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

evaluation = model.evaluate_generator(test_generator, steps=test_generator.samples // test_generator.batch_size)


print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

<ipython-input-28-1eb6dad4c313>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation = model.evaluate_generator(test_generator, steps=test_generator.samples // test_generator.batch_size)


Test Loss: 2.6382193565368652
Test Accuracy: 0.31990131735801697


# Demonstrating how the model performed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


class_labels = list(train_generator.class_indices.keys())


num_images = 20


num_rows = (num_images + 4) // 5  # 4 columns
num_cols = 5


fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 4 * num_rows))


if num_rows == 1:
    axes = axes.reshape(1, -1)
elif num_cols == 1:
    axes = axes.reshape(-1, 1)

for i in range(len(test_generator)):

    batch_images, batch_labels = test_generator[i]

    predictions = model.predict(batch_images)


    for j in range(len(batch_images)):

        predicted_label_index = np.argmax(predictions[j])

        true_label_index = np.argmax(batch_labels[j])

        predicted_label = class_labels[predicted_label_index]
        true_label = class_labels[true_label_index]

        index = i * len(batch_images) + j
        if index < num_images:
            row = index // num_cols
            col = index % num_cols
            axes[row, col].imshow(batch_images[j])
            axes[row, col].set_title(f"Predicted: {predicted_label}, True: {true_label}")
            axes[row, col].axis('off')


plt.tight_layout()
plt.show()


# Resnet


In [ ]:

from tensorflow.keras.applications import ResNet50


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(len(class_data_lists), activation='softmax')(x)

resnet = Model(inputs=base_model.input, outputs=x)


resnet.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
resnet.summary()


94765736/94765736 [==============================] - 0s 0us/step


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:

history = resnet.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size)





<ipython-input-39-b0b2d5617731>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = resnet.fit_generator(


Epoch 1/20
153/153 [==============================] - 83s 504ms/step - loss: 5.2640 - accuracy: 0.2496 - val_loss: 2.6277 - val_accuracy: 0.3207
Epoch 2/20
153/153 [==============================] - 79s 514ms/step - loss: 2.6340 - accuracy: 0.3132 - val_loss: 2.5828 - val_accuracy: 0.3207
Epoch 3/20
153/153 [==============================] - 77s 505ms/step - loss: 2.5827 - accuracy: 0.3216 - val_loss: 2.5321 - val_accuracy: 0.3248
Epoch 4/20
153/153 [==============================] - 76s 498ms/step - loss: 2.5210 - accuracy: 0.3212 - val_loss: 2.6169 - val_accuracy: 0.2936
Epoch 5/20
153/153 [==============================] - 76s 497ms/step - loss: 2.5278 - accuracy: 0.3208 - val_loss: 2.4677 - val_accuracy: 0.3273
Epoch 6/20
153/153 [==============================] - 81s 527ms/step - loss: 2.4776 - accuracy: 0.3249 - val_loss: 2.4841 - val_accuracy: 0.3265
Epoch 7/20
153/153 [==============================] - 76s 499ms/step - loss: 2.4135 - accuracy: 0.3266 - val_loss: 2.4184 - val_ac

# Inception V3

In [ ]:
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(20, activation='softmax')(x)

inception = Model(inputs=base_model.input, outputs=x)
inception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inception.summary()


87910968/87910968 [==============================] - 1s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)   

In [ ]:
history = inception.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size)


<ipython-input-41-52cee4fbce0b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = inception.fit_generator(


Epoch 1/20
153/153 [==============================] - 85s 503ms/step - loss: 4.0044 - accuracy: 0.4047 - val_loss: 1.8859 - val_accuracy: 0.4572
Epoch 2/20
153/153 [==============================] - 75s 493ms/step - loss: 1.5860 - accuracy: 0.5305 - val_loss: 1.6950 - val_accuracy: 0.5345
Epoch 3/20
153/153 [==============================] - 74s 487ms/step - loss: 1.4063 - accuracy: 0.5582 - val_loss: 1.6706 - val_accuracy: 0.5090
Epoch 4/20
153/153 [==============================] - 73s 479ms/step - loss: 1.2911 - accuracy: 0.5830 - val_loss: 1.6223 - val_accuracy: 0.5362
Epoch 5/20
153/153 [==============================] - 75s 492ms/step - loss: 1.2333 - accuracy: 0.5957 - val_loss: 1.7290 - val_accuracy: 0.4811
Epoch 6/20
153/153 [==============================] - 79s 514ms/step - loss: 1.1986 - accuracy: 0.5906 - val_loss: 1.7895 - val_accuracy: 0.4959
Epoch 7/20
153/153 [==============================] - 75s 488ms/step - loss: 1.1488 - accuracy: 0.6064 - val_loss: 1.6995 - val_ac

# Showing top 10 images

In [ ]:
def custom_decode_predictions(predictions, top=1, class_labels=class_data_lists):
    top_indices = tf.math.top_k(predictions, k=top).indices.numpy()[0]
    top_classes = [list(class_labels.keys())[i] for i in top_indices]
    top_confidences = predictions[0][top_indices]

    return list(zip(top_classes, top_confidences))

In [ ]:
for class_name in class_data_lists.keys():
    print(f"\nTop 10 ranked photos for class: {class_name}")

    class_folder_path = os.path.join(test_path, class_name)
    class_image_files = os.listdir(class_folder_path)[:10]  # Consider only the first 10 images

    for image_file in class_image_files:
        img_path = os.path.join(class_folder_path, image_file)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

        predictions = model.predict(img_array)
        decoded_predictions = custom_decode_predictions(predictions, top=1)

        print(f"{image_file}: {decoded_predictions[0][0]} - {decoded_predictions[0][1]:.2f}")


Top 10 ranked photos for class: Aeroplane
1/1 [==============================] - 1s 1s/step
2008_002551.jpg: Boat - 0.96
1/1 [==============================] - 0s 17ms/step
2008_007480.jpg: Car - 1.00
1/1 [==============================] - 0s 17ms/step
2008_008278.jpg: Aeroplane - 1.00
1/1 [==============================] - 0s 18ms/step
2008_003905.jpg: Aeroplane - 1.00
1/1 [==============================] - 0s 25ms/step
2008_007883.jpg: Bus - 1.00
1/1 [==============================] - 0s 18ms/step
2008_008471.jpg: Bird - 1.00
1/1 [==============================] - 0s 19ms/step
2008_003189.jpg: Train - 1.00
1/1 [==============================] - 0s 20ms/step
2008_003261.jpg: Aeroplane - 1.00
1/1 [==============================] - 0s 20ms/step
2008_002221.jpg: Aeroplane - 1.00
1/1 [==============================] - 0s 23ms/step
2008_008546.jpg: Bird - 0.67

Top 10 ranked photos for class: Bicycle
1/1 [==============================] - 0s 23ms/step
2008_007510.jpg: Person - 1.00
1/1 [=

In [ ]:
import matplotlib.pyplot as plt

for class_name in class_data_lists.keys():
    print(f"\nTop 3 ranked photos for class: {class_name}")

    class_folder_path = os.path.join(test_path, class_name)
    class_image_files = os.listdir(class_folder_path)[:3]
    for image_file in class_image_files:
        img_path = os.path.join(class_folder_path, image_file)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = tf.expand_dims(img_array, 0)

        predictions = model.predict(img_array)
        decoded_predictions = custom_decode_predictions(predictions, top=1)


        plt.imshow(img)
        plt.title(f"Predicted class: {decoded_predictions[0][0]} - Confidence: {decoded_predictions[0][1]:.2f}")
        plt.axis('off')
        plt.show()



# Evaluation on class bases

In [ ]:
from sklearn.metrics import classification_report


def calculate_metrics(model, test_generator):

    y_pred = model.predict(test_generator)


    y_pred_labels = np.argmax(y_pred, axis=1)


    y_true = test_generator.classes


    class_labels = list(test_generator.class_indices.keys())


    report = classification_report(y_true, y_pred_labels, target_names=class_labels)

    return report


vgg_metrics = calculate_metrics(model, test_generator)
resnet_metrics = calculate_metrics(resnet, test_generator)
inception_metrics = calculate_metrics(inception, test_generator)


print("VGG Metrics:")
print(vgg_metrics)

print("\nResNet Metrics:")
print(resnet_metrics)

print("\nInception Metrics:")
print(inception_metrics)


39/39 [==============================] - 8s 192ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


39/39 [==============================] - 8s 181ms/step
VGG Metrics:
              precision    recall  f1-score   support

   Aeroplane       0.02      0.02      0.02        48
     Bicycle       0.00      0.00      0.00        39
        Bird       0.07      0.10      0.08        61
        Boat       0.00      0.00      0.00        42
      Bottle       0.03      0.02      0.02        49
         Bus       0.00      0.00      0.00        20
         Car       0.04      0.04      0.04        94
         Cat       0.07      0.05      0.05        66
       Chair       0.08      0.10      0.09        71
         Cow       0.11      0.07      0.08        15
 Diningtable       0.00      0.00      0.00        21
         Dog       0.06      0.06      0.06        78
       Horse       0.00      0.00      0.00        40
   Motorbike       0.00      0.00      0.00        41
      Person       0.34      0.36      0.35       401
 Pottedplant       0.03      0.06      0.04        36
       Sheep 